In [ ]:
!pip install voila matplotlib seaborn pandas numpy requests

In [10]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Function to fetch real-time USDA milk component prices
def get_usda_prices():
    url = "https://www.ams.usda.gov/mnreports/dymclassprices.txt"  # USDA report URL
    try:
        response = requests.get(url)
        text = response.text
        lines = text.split("\n")
        prices = {"Butterfat": 2.95, "Protein": 2.33, "Other Solids": 0.54}  # Default values
        
        for line in lines:
            if "Butterfat" in line:
                prices["Butterfat"] = float(line.split()[-1])
            elif "Protein" in line:
                prices["Protein"] = float(line.split()[-1])
            elif "Other Solids" in line:
                prices["Other Solids"] = float(line.split()[-1])
        
        return prices
    except:
        return {"Butterfat": 2.95, "Protein": 2.33, "Other Solids": 0.54}  # Return defaults if API fails

# Fetch updated milk prices
milk_prices = get_usda_prices()

# Create interactive input widgets
dmi = widgets.FloatText(value=55.0, description="DMI (lbs/day):")
milk_yield = widgets.FloatText(value=90.0, description="Milk Yield (lbs/day):")
milk_fat = widgets.FloatText(value=4.1, description="Milk Fat (%):")
milk_protein = widgets.FloatText(value=3.2, description="Protein (%):")
milk_lactose = widgets.FloatText(value=4.85, description="Lactose (%):")
carbon_credit_price = widgets.FloatText(value=40.0, description="Carbon Credit ($/tonne):")

# Dropdown for feed additives
feed_additive_type = widgets.Dropdown(
    options=["3-NOP", "Essential Oils", "Tannins", "Yeast Culture"],
    value="3-NOP",
    description="Feed Additive:"
)

# Button to run calculations
calc_button = widgets.Button(description="Calculate ROI", button_style='success')

# Display widgets
display(dmi, milk_yield, milk_fat, milk_protein, milk_lactose, carbon_credit_price, feed_additive_type, calc_button)

# Function to perform calculations & generate visuals
def calculate_roi(_):
    feed_additive_costs = {"3-NOP": 0.05, "Essential Oils": 0.03, "Tannins": 0.04, "Yeast Culture": 0.02}
    feed_additive_cost = feed_additive_costs[feed_additive_type.value]

    # Compute Milk Component Revenues
    fat_revenue = milk_yield.value * (milk_fat.value / 100) * milk_prices["Butterfat"]
    protein_revenue = milk_yield.value * (milk_protein.value / 100) * milk_prices["Protein"]
    other_solids_revenue = milk_yield.value * (milk_lactose.value / 100) * milk_prices["Other Solids"]
    total_milk_revenue = fat_revenue + protein_revenue + other_solids_revenue

    # Ration Cost Calculation
    ration_cost_per_lb = 0.18
    ration_cost_per_cow = ration_cost_per_lb * dmi.value

    # IOFC Calculation
    iofc = total_milk_revenue - ration_cost_per_cow

    # Methane Mitigation (assuming 5% reduction)
    ch4_emissions = 457  # Default CH4 emissions in g/d
    ch4_reduction = ch4_emissions * -0.05  # 5% reduction
    co2_eq_mitigation = ch4_reduction * 28  # Convert CH4 to CO2 equivalent
    carbon_credit_revenue = (co2_eq_mitigation * carbon_credit_price.value) / 1_000_000  # Convert g to tonnes

    # Net Return & ROI Calculation
    net_return = (carbon_credit_revenue + total_milk_revenue - ration_cost_per_cow) - feed_additive_cost
    roi = net_return / feed_additive_cost

    # Breakeven Analysis
    breakeven_price_carbon_credit = carbon_credit_price.value + (net_return / co2_eq_mitigation * 1_000_000) if co2_eq_mitigation != 0 else 0
    breakeven_feed_additive_price = total_milk_revenue - ration_cost_per_cow + ch4_reduction

    # Summary Table
    results_df = pd.DataFrame({
        "Metric": [
            "Milk Revenue ($/cow/day)", "Ration Cost ($/cow/day)", "IOFC ($/cow/day)",
            "CH4 Reduction (g/d)", "CO2 Equivalent Mitigation (g/d)", "Carbon Credit Revenue ($/cow/day)",
            "Net Return Feed Additive ($/cow/day)", "ROI (%)", "Breakeven Feed Additive Price ($/cow/day)"
        ],
        "Value": [
            total_milk_revenue, ration_cost_per_cow, iofc, ch4_reduction, co2_eq_mitigation,
            carbon_credit_revenue, net_return, roi, breakeven_feed_additive_price
        ]
    })

    display(results_df)

    # Create Plots
    fig, axes = plt.subplots(3, 1, figsize=(7, 10))

    # ROI vs Carbon Value
    carbon_values = np.linspace(0, 400, 10)
    roi_values = roi + (carbon_values * 0.02)  # Example trend
    axes[0].plot(carbon_values, roi_values, label="ROI for Your Inputs", color='blue')
    axes[0].axhline(y=0, color='red', linestyle='dashed')  # Zero ROI line
    axes[0].set_xlabel("Carbon value, $/ton")
    axes[0].set_ylabel("Return on investment (%)")
    axes[0].legend()
    axes[0].set_title("ROI vs. Carbon Value")

    # CH4 Emissions Bar Chart
    sns.barplot(x=["Original", "Mean Scenarios", "Your Inputs"], y=[457, 328, ch4_reduction], ax=axes[1], palette="Blues")
    axes[1].set_title("Enteric CH4 Emissions (g/d)")
    axes[1].set_ylabel("CH4 (g/d)")

    # Breakeven Feed Additive Price
    sns.barplot(x=["Original", "Mean", "Your Inputs"], y=[0.00, 0.08, breakeven_feed_additive_price], ax=axes[2], palette="Blues")
    axes[2].set_title("Breakeven Feed Additive Price ($/cow/day)")
    axes[2].set_ylabel("Price ($)")

    plt.tight_layout()
    plt.show()

    # Save results to CSV
    results_df.to_csv("dairy_roi_results.csv", index=False)
    display(HTML("<b>Results saved to <a href='dairy_roi_results.csv' download>dairy_roi_results.csv</a></b>"))

# Link button to function
calc_button.on_click(calculate_roi)

FloatText(value=55.0, description='DMI (lbs/day):')

FloatText(value=90.0, description='Milk Yield (lbs/day):')

FloatText(value=4.1, description='Milk Fat (%):')

FloatText(value=3.2, description='Protein (%):')

FloatText(value=4.85, description='Lactose (%):')

FloatText(value=40.0, description='Carbon Credit ($/tonne):')

Dropdown(description='Feed Additive:', options=('3-NOP', 'Essential Oils', 'Tannins', 'Yeast Culture'), value=…

Button(button_style='success', description='Calculate ROI', style=ButtonStyle())